In [ ]:
from IPython.display import display, HTML
display(HTML("<style>:root { --jp-notebook-max-width: 100% !important; }</style>"))


import importlib, numpy as np
from sys import path
import time, os, copy

from sunpy.map import Map
from sunpy.net import Fido, vso, attrs as a
from sunpy.time import TimeRange
from ndcube import NDCube, NDCubeSequence, NDCollection
from astropy import wcs

import astropy.units as u
import matplotlib.pyplot as plt

%matplotlib widget


plt.rcParams.update({'font.size': 18}) # Make the fonts in figures big enough for papers
plt.rcParams.update({'figure.figsize':[27,15]})
plt.rcParams.update({'image.origin':'lower'})
np.set_printoptions(linewidth=128)

base_path = os.getcwd()

import EMToolKit.EMToolKit as emtk
# from EMToolKit.visualization.dashboard import dashboard_figure
from EMToolKit.instruments.aia import load_from_paths, aia_wrapper
from EMToolKit.instruments.xrt import xrt_wrapper
from EMToolKit.algorithms.sparse_em_wrapper import sparse_em_wrapper
from EMToolKit.algorithms.simple_reg_dem_wrapper import simple_reg_dem_wrapper
from EMToolKit.algorithms.sparse_nlmap_dem_wrapper import sparse_nlmap_dem_wrapper

In [ ]:
import EMToolKit
import EMToolKit.visualization
importlib.reload(EMToolKit)
importlib.reload(EMToolKit.visualization)
# importlib.reload(EMToolKit.visualization.dashboard)
import EMToolKit.EMToolKit as emtk
# from EMToolKit.visualization.dashboard import dashboard_figure
em_collection=0

In [ ]:
daystr = '20100725'
date='2010/07/25 17:58:44'
data_path = os.path.join(base_path,'../data',daystr)

xl, yl, dx, dy = 240*u.arcsec, -525*u.arcsec, 270*u.arcsec, 240*u.arcsec
#xl, yl, dx, dy = 240*u.arcsec, -525*u.arcsec, 135*u.arcsec, 120*u.arcsec

if not os.path.exists(data_path):
    os.makedirs(data_path)
paths = [data_path + pth for pth in os.listdir(data_path)
         if (os.path.isfile(os.path.join(data_path, pth)) and ".fits" in pth)]
print(f"Searching for images from {date}...")

redownload = False
if len(paths) < 7 or redownload:
# Commands for initial data download. Comment out once that's successful.
# VSO can sometimes be a bit flakey here, in my experience, may require multiple tries:
    print(f"Only {len(paths)} of 7 images found, searching FIDO...")
    dl_paths = []
    passbands = np.array([94,131,171,193,211,335])*u.angstrom

    # Combine the wavelength queries using the | operator
    wavelength_query = a.Wavelength(passbands[0])
    for band in passbands[1:]:
        wavelength_query |= a.Wavelength(band)

    qry = Fido.search(a.Time(TimeRange(date,11.8*u.s)),a.Instrument('AIA')|a.Instrument('XRT'),wavelength_query)

    print(f"Downloading {len(qry)} images from AIA...")
    Fido.fetch(qry,path=data_path, max_conn=len(passbands)+1)

    qry = Fido.search(a.Time(TimeRange(date,4*u.s)), a.Instrument('XRT'))
    print(f"Downloading {len(qry)} images from XRT...")
    Fido.fetch(qry,path=data_path, max_conn=len(passbands)+1)
else:
    print(f"Found {len(paths)} images, skipping download.")


paths = sorted([os.path.abspath(os.path.join(data_path,pth)) for pth in os.listdir(data_path)
         if (os.path.isfile(os.path.join(data_path, pth)) and ".fits" in pth)])

channel_names = sorted(np.array(['AIA94_THIN', 'AIA131_THIN', 'AIA171_THIN', 'AIA193_THIN', 'AIA211_THIN', 'AIA335_THIN', 'XRT']))

hinode_path = paths[-1]


In [ ]:
aiapaths = paths[0:6]

aiamaps = load_from_paths(paths[0:6],xl=xl,yl=yl,dx=dx,dy=dy)
# We're loading these separately so that we can apply an offset to the XRT data -- there appears
# to be a significant misalignment based on manual inspection and testing:
#xrt_offsets = [-2*u.arcsec, -11*u.arcsec]
#xrt_offsets = [-7*u.arcsec, -23*u.arcsec]
xrt_offsets = [-5*u.arcsec, -20*u.arcsec]
# xrtmap = load_from_paths(paths[-1],xl=xl+xrt_offsets[0],yl=yl+xrt_offsets[1],dx=dx,dy=dy)[1]
xrtmap = load_from_paths([[paths[0]],hinode_path],xl=xl+xrt_offsets[0],yl=yl+xrt_offsets[1],dx=dx,dy=dy)[1]

xrtmap.meta['crval1'] -= xrt_offsets[0].value
xrtmap.meta['crval2'] -= xrt_offsets[1].value

[xrtmaps,xrterrs,xrt_trlogts,xrt_tresps] = xrt_wrapper([xrtmap])
[maps,errs,trlogts,tresps] = aia_wrapper(aiamaps)
maps.append(xrtmaps[0])
errs.append(xrterrs[0])
trlogts.append(xrt_trlogts[0])
tresps.append(xrt_tresps[0])

logt = np.linspace(5.5,7.5,41)
for i in range(0,len(tresps)):
	tresps[i] = np.interp(logt,trlogts[i],tresps[i])
	trlogts[i] = copy.deepcopy(logt)

# Normalizing the response functions makes them much easier for the solver to handle.
datmax_nominal = 1.0e4 # The nominal maximum of the data
overall_norm = datmax_nominal/np.max(np.array(tresps))
norms = np.max(np.array(tresps),axis=0)/np.max(np.array(tresps))/overall_norm

In [ ]:
# Make data sequence from AIA & XRT data:
datasequence = emtk.em_data(maps,errs,trlogts,tresps)

# Create em_collection with AIA & XRT data:
em_collection = emtk.em_collection(datasequence)

In [ ]:
# Show some of the AIA channels and their temperature responses for illustration:
plt_emmax = 5.0e28
gfac = 1.0/2.2

fig, axs = plt.subplots(2, 3)
axs[0,0].imshow(np.clip(maps[1].data/np.max(tresps[1]),0,plt_emmax)**gfac,cmap=plt.get_cmap('gray'))
axs[0,0].set(title='AIA 193 Angstrom')
axs[1,0].plot(trlogts[1],tresps[1]/1.0e-27)
axs[1,0].set(title='AIA 193 Temperature Response',xlabel='Temperature (dB Kelvin)',ylabel='Response (DN/s/(10$^9$ cm$^{-3})^2$/Mm)')

axs[0,1].imshow(np.clip(maps[2].data/np.max(tresps[2]),0,plt_emmax)**gfac,cmap=plt.get_cmap('gray'))
axs[0,1].set(title='AIA 335 Angstrom')
axs[1,1].plot(trlogts[2],tresps[2]/1.0e-27)
axs[1,1].set(title='AIA 335 Temperature Response',xlabel='Temperature (dB Kelvin)',ylabel='Response (DN/s/(cm$^{-3})^2$/Mm)')

axs[0,2].imshow(np.clip(maps[5].data/np.max(tresps[5]),0,plt_emmax)**gfac,cmap=plt.get_cmap('gray'))
axs[0,2].set(title='AIA 94 Angstrom')
axs[1,2].plot(trlogts[5],tresps[5]/1.0e-27)
axs[1,2].set(title='AIA 94 Temperature Response',xlabel='Temperature (dB Kelvin)',ylabel='Response (DN/s/(cm$^{-3})^2$/Mm)')

In [ ]:

# Make DEM sequence from the DEM and add it to the collection:
multi_out = sparse_nlmap_dem_wrapper(datasequence, wrapargs={'norms':norms, 'overall_norm':overall_norm})
multi_em_demsequence = emtk.dem_model(*multi_out)
em_collection.add_model(multi_em_demsequence)

# Unpack the output
multi_coeffs, multi_logts, multi_bases, multi_coords, multi_algorithm, multi_em_wrapper = multi_out

In [ ]:
# # print(type((collect)))
# # collect_dict = dict(collect)
# # print(collect_dict.keys())
# # print(collect_dict["data"])


# import asdf
# from ndcube import NDCollection

# # Your NDCollection
# my_ndcollection = collect

# # Create a new ASDF file
# with asdf.AsdfFile() as af:
#     # Iterate over items in the NDCollection and add them to the ASDF file
#     # print(my_ndcollection['models'])

#     af['data'] = []
#     af['meta_keys'] = []
#     af['meta_values'] = []
#     af['methods'] = []
#     for img in my_ndcollection['data']:
#         af['data'].append(img.data)
#         img_meta = copy.copy(img.meta)
#         # schema = img_meta["schema"]
#         # del img_meta["schema"]
#         # print (img_meta)
#         af['meta_keys'].append(img_meta.keys())
#         af['meta_values'].append(img_meta.values())

#     for mthd in my_ndcollection['models']:
#         af['methods'].append(mthd)
#         # af['method_meta'].append(([key, value] for key, value in mthd.meta.items()))

#     for key, ndcube in my_ndcollection.items():
#         # Here, 'key' can be used to uniquely identify each NDCube in the collection
#         # print (dir(ndcube))
#         # af[key] = {
#         #     'data': ,
#         #     # 'meta': ndcube.meta
#         # }
#         pass
#         # print(key)
#         # print((my_ndcollection[key]))
#     # Write the ASDF file to disk
#     af.write_to(f'{data_path}/my_ndcollection.fits')


# # print(collect_dict['data'][0].wcs)

In [ ]:
multi_synthmaps = [em_collection.synthesize_map(map) for map in em_collection.data()]
multi_synthdata = [map for map in multi_synthmaps]

In [ ]:
def calc_resids(synthdata, em_collection):# Calculate the residuals and Chi squared:
	ndata = len(synthdata)
	resids = []
	datasequence = em_collection.data()
	chi2 = 0
	[nx,ny] = datasequence[0].data.shape
	for seq in datasequence: [nx,ny] = [np.min([seq.data.shape[0],nx]),np.min([seq.data.shape[1],ny])]
	for i in range(0,ndata):
		resids.append(((synthdata[i].data-datasequence[i].data)/datasequence[i].uncertainty.array)**2)
		chi2 += np.mean(resids[i])/ndata
	return resids, chi2

#spars_resids, spars_chi2 = calc_resids(spars_synthdata,em_collection)
#simpl_resids, simpl_chi2 = calc_resids(simpl_synthdata,em_collection)
multi_resids, multi_chi2 = calc_resids(multi_synthdata,em_collection)
print("Multi-instrument Chi squared = ",multi_chi2)
#print('simple_reg_dem Chi squared = ',simpl_chi2, 'spars_em Chi squared = ',spars_chi2)

In [ ]:
# Plot Residuals:
fig = plt.figure(figsize=[20,12])
plt.suptitle('Residuals for '+multi_algorithm)
for i in range(0,7):
    ax1 = fig.add_subplot(2,4,i+1)
    ax1.imshow(multi_resids[i],vmin=0,vmax=5)
    ax1.set(title=em_collection.data()[i].meta['channel'])

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets


# Get the most recent version from the tool kit:
from EMToolKit.visualization.dashboard import dashboard_object
importlib.reload(EMToolKit.visualization.dashboard)
from EMToolKit.visualization.dashboard import dashboard_object

# Create the dashboard:
dash = dashboard_object(em_collection)
dash.display()


# class dashboard_object(object):
#     def __init__(self,em_collection):
#         self.emc = em_collection

#     def widgwrap(self, xpt, ypt, rtemp, gtemp, btemp, sigma, algorithm):
#         dashboard_figure(self.emc, plotpoint=[xpt,ypt], temperatures=[rtemp,gtemp,btemp], sigmas=sigma, algorithm=algorithm)

In [ ]:
# importlib.reload(EMToolKit.visualization.dashboard)
# from EMToolKit.visualization.dashboard import dashboard_figure
# dash = dashboard_object(em_collection)

In [ ]:
# [nx,ny] = dash.emc.collection[dash.emc.collection['models'][0]][0].data.shape
# xpt=widgets.IntSlider(min=0, max=nx, value=10, step=1, description='xpt', continuous_update=False)
# ypt=widgets.IntSlider(min=0, max=ny, value=100, step=1, description='ypt', continuous_update=False)
# rtemp=widgets.FloatSlider(min=5, max=7, value=5.8, step=0.05, description='rtemp', continuous_update=False)
# gtemp=widgets.FloatSlider(min=5, max=7, value=6.1, step=0.05, description='gtemp', continuous_update=False)
# btemp=widgets.FloatSlider(min=5, max=7, value=6.4, step=0.05, description='btemp', continuous_update=False)
# sigma=widgets.FloatSlider(min=0.025, max=0.5, value=0.125, step=0.01, description='sigma', continuous_update=False)
# algorithm=widgets.Dropdown(options=dash.emc.collection['models'], description='algorithm', continuous_update=False)
# ui = widgets.HBox([xpt,ypt,rtemp,gtemp,btemp,sigma,algorithm])
# out = widgets.interactive_output(dash.widgwrap, {'xpt':xpt,'ypt':ypt,'rtemp':rtemp,'gtemp':gtemp,'btemp':btemp,'sigma':sigma,'algorithm':algorithm})
# display(ui,out)